In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import sys
print(sys.executable)
sys.path.insert(1, '../src/')
from config import raw_data_path, univariate_data_path, processed_data_path, models_path
from preprocessing_modules import create_time_windows_with_labels, create_time_windows_with_metadata
from skopt import gp_minimize
from skopt.space import Real, Integer, Categorical
from skopt.utils import use_named_args
from skopt import gp_minimize
from skopt.space import Real, Integer
from skopt.utils import use_named_args
import matplotlib.pyplot as plt
from FCMAE_model import FCMAE


/home/nwertheim/miniconda3/bin/python
/home/nwertheim/miniconda3/bin/python


In [2]:
# Define FCMAE Hyperparameters:
batch_size = 16
num_blocks = 5
kernel_size = 7
base_dim = 32
learning_rate = 0.00016938000495408888
input_dimension = 512

In [3]:

import torch
import numpy as np
import os
from torch.utils.data import TensorDataset, DataLoader, Subset
from sklearn.model_selection import StratifiedKFold
from sklearn.utils.class_weight import compute_class_weight
import optuna
from fastai.learner import Learner
from fastai.data.core import DataLoaders
from fastai.metrics import accuracy
from tsai.models.ResNet import ResNet
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from fastai.optimizer import SGD
# Load training data
train_dir = os.path.join(univariate_data_path, "target_univariate_no_PCA_train_2.npy")
train_data = np.load(train_dir, allow_pickle=True)
train_windows, train_labels, train_meta = create_time_windows_with_metadata(train_data)

# Convert to tensors
train_windows_tensor = torch.tensor(train_windows, dtype=torch.float32).unsqueeze(1)  # shape: (N, 1, 12000)
train_labels_tensor = torch.tensor(train_labels, dtype=torch.long)

# Compute class weights
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(train_labels), y=train_labels)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32).to(device)

# Prepare dataset
full_dataset = TensorDataset(train_windows_tensor, train_labels_tensor)

/home/nwertheim/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:



# Load the pre-trained encoder
path = os.path.join(models_path, 'FCMAE_encoder_no_PCA_gpu_normalized_correct.pth')

# Load the full FCMAE model
fcmae = FCMAE(in_channels=1, base_dim=base_dim, num_blocks=num_blocks, kernel_size=kernel_size)


# Load the saved weights into the full model
state_dict = torch.load(path)
fcmae.load_state_dict(state_dict)

# Extract only the encoder from the FCMAE model
encoder = fcmae.encoder
# Set the encoder to evaluation mode and freeze the encoder parameters
encoder.eval()
for param in encoder.parameters():
    param.requires_grad = False  # Freeze the encoder

print(encoder)

Sequential(
  (0): Conv1d(1, 32, kernel_size=(7,), stride=(2,), padding=(3,))
  (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): Conv1d(32, 64, kernel_size=(7,), stride=(2,), padding=(3,))
  (4): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (5): ReLU()
  (6): Conv1d(64, 128, kernel_size=(7,), stride=(2,), padding=(3,))
  (7): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (8): ReLU()
  (9): Conv1d(128, 256, kernel_size=(7,), stride=(2,), padding=(3,))
  (10): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (11): ReLU()
  (12): Conv1d(256, 512, kernel_size=(7,), stride=(2,), padding=(3,))
  (13): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (14): ReLU()
)


In [5]:
'''SGD OPTIMIZER'''
from fastai.learner import Metric
from sklearn.metrics import average_precision_score

# Custom Average Precision Metric
class AveragePrecision(Metric):
    def __init__(self):
        self.pred = []
        self.target = []

    def reset(self):
        self.pred, self.target = [], []

    def accumulate(self, learn):
        preds = learn.pred.detach().softmax(dim=-1)[:, 1]
        targs = learn.y.detach()
        self.pred.append(preds.cpu())
        self.target.append(targs.cpu())

    @property
    def value(self):
        preds = torch.cat(self.pred).numpy()
        targs = torch.cat(self.target).numpy()
        return average_precision_score(targs, preds)

    @property
    def name(self): return "avg_precision"

def objective(trial):
    lr = trial.suggest_loguniform('lr', 1e-6, 1e-3)
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    avg_precisions = []

    for fold, (train_idx, val_idx) in enumerate(skf.split(train_windows_tensor, train_labels_tensor)):
        train_subset = Subset(full_dataset, train_idx)
        val_subset = Subset(full_dataset, val_idx)

        train_loader = DataLoader(train_subset, batch_size=64, shuffle=True)
        val_loader = DataLoader(val_subset, batch_size=64, shuffle=False)
        dls = DataLoaders(train_loader, val_loader)

        class FCMAEClassifier(nn.Module):
            def __init__(self, encoder):
                super().__init__()
                self.encoder = encoder
                self.classifier = ResNet(input_dimension, 2)

            def forward(self, x):
                with torch.no_grad():
                    x = self.encoder(x)
                return self.classifier(x)

        model = FCMAEClassifier(encoder).to(device)
        loss_func = torch.nn.CrossEntropyLoss(weight=class_weights_tensor)

        learn = Learner(
            dls, model,
            loss_func=loss_func,
            metrics=AveragePrecision()
        )

        learn.fit_one_cycle(5, lr_max=lr, wd=0.01)
        ap = learn.validate()[1]  # [loss, avg_precision]
        avg_precisions.append(ap)

    return np.mean(avg_precisions)  # maximize this

# Run Optuna study
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=25)

print("Best trial:")
print(study.best_trial)


# Save the study
import joblib
joblib.dump(study, "FCMAE_ResNet_optuna_ADAM.pkl")


[I 2025-05-12 14:06:41,814] A new study created in memory with name: no-name-65f4019c-a6b9-4bce-b941-a1b2c4cabd72


epoch,train_loss,valid_loss,avg_precision,time
0,0.682479,0.669786,0.741218,00:01
1,0.664588,0.657001,0.771692,00:01
2,0.652822,0.642815,0.814549,00:01
3,0.643259,0.666175,0.762066,00:01
4,0.631973,0.628280,0.822387,00:01


epoch,train_loss,valid_loss,avg_precision,time
0,0.675752,0.647285,0.746773,00:01
1,0.660001,0.668550,0.740409,00:01
2,0.646501,0.673481,0.763233,00:01
3,0.636771,0.630328,0.814501,00:01
4,0.628585,0.642506,0.790249,00:01


epoch,train_loss,valid_loss,avg_precision,time
0,0.672482,0.656903,0.727802,00:01
1,0.656515,0.651662,0.798176,00:01
2,0.644850,0.639926,0.800558,00:01
3,0.638851,0.642184,0.795441,00:01
4,0.632938,0.633786,0.803695,00:01


epoch,train_loss,valid_loss,avg_precision,time
0,0.677369,0.666570,0.723752,00:01
1,0.663362,0.632177,0.752242,00:01
2,0.650607,0.613674,0.815671,00:01
3,0.641302,0.613758,0.820454,00:01
4,0.633426,0.648556,0.751857,00:01


...

epoch,train_loss,valid_loss,avg_precision,time
0,0.676189,0.770296,0.754004,00:01
1,0.658174,0.704054,0.755773,00:01
2,0.651718,0.630647,0.805063,00:01
3,0.642256,0.630211,0.781127,00:01
4,0.630280,0.633987,0.782524,00:01


epoch,train_loss,valid_loss,avg_precision,time
0,0.672707,0.865398,0.720028,00:01
1,0.660637,0.682444,0.748668,00:01
2,0.649399,0.629212,0.802606,00:01
3,0.640535,0.649397,0.751455,00:01
4,0.627013,0.626823,0.796882,00:01


[I 2025-05-12 14:18:15,227] Trial 24 finished with value: 0.7736846465322472 and parameters: {'lr': 0.0004331104085069829}. Best is trial 8 with value: 0.8044351288005522.


Best trial:
FrozenTrial(number=8, state=1, values=[0.8044351288005522], datetime_start=datetime.datetime(2025, 5, 12, 14, 10, 23, 352330), datetime_complete=datetime.datetime(2025, 5, 12, 14, 10, 51, 124106), params={'lr': 0.0001582887649804575}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'lr': FloatDistribution(high=0.001, log=True, low=1e-06, step=None)}, trial_id=8, value=None)


['FCMAE_ResNet_optuna_ADAM.pkl']

In [9]:
print("Best trial:")
print(study.best_trial)


Best trial:
FrozenTrial(number=8, state=1, values=[0.8044351288005522], datetime_start=datetime.datetime(2025, 5, 12, 14, 10, 23, 352330), datetime_complete=datetime.datetime(2025, 5, 12, 14, 10, 51, 124106), params={'lr': 0.0001582887649804575}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'lr': FloatDistribution(high=0.001, log=True, low=1e-06, step=None)}, trial_id=8, value=None)


In [7]:
# '''MY DATA'''
# train_dir = os.path.join(univariate_data_path, "target_univariate_no_PCA_train2.npy")
# train_data = np.load(train_dir, allow_pickle=True)
# # train_data = pd.DataFrame(train_data)
# test_dir = os.path.join(univariate_data_path, "target_univariate_no_PCA_test2.npy")
# test_data = np.load(test_dir, allow_pickle=True)
# # test_data = pd.DataFrame(test_data)
# print(len(train_data))
# print(len(test_data))
# print(train_data[0])
# from torch.utils.data import DataLoader, TensorDataset, random_split
# from sklearn.utils.class_weight import compute_class_weight

# from collections import Counter

# # Example dataset with windows and labels
# train_windows, train_labels = create_time_windows_with_labels(train_data)
# test_windows, test_labels = create_time_windows_with_labels(test_data)

# # Count label distribution
# train_label_counts = Counter(train_labels)
# test_label_counts = Counter(test_labels)

# # Print the counts
# print("Train label distribution:")
# print(f"  Term (0): {train_label_counts[0]}")
# print(f"  Preterm (1): {train_label_counts[1]}")

# print("\nTest label distribution:")
# print(f"  Term (0): {test_label_counts[0]}")
# print(f"  Preterm (1): {test_label_counts[1]}")

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# # Convert to tensors
# train_windows_tensor = torch.tensor(train_windows, dtype=torch.float32)
# train_labels_tensor = torch.tensor(train_labels, dtype=torch.float32)
# test_windows_tensor = torch.tensor(test_windows, dtype=torch.float32)
# test_labels_tensor = torch.tensor(test_labels, dtype=torch.float32)

# train_windows_tensor = train_windows_tensor.unsqueeze(1)  # (1071, 1, 12000)
# test_windows_tensor = test_windows_tensor.unsqueeze(1)    # (899, 1, 12000)

# train_labels_tensor = train_labels_tensor.long()
# test_labels_tensor = test_labels_tensor.long()


# print(train_labels_tensor.shape)

# train_dataset = TensorDataset(train_windows_tensor, train_labels_tensor)
# test_dataset = TensorDataset(test_windows_tensor, test_labels_tensor)

# train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)



In [8]:
# '''CLASS WEIGHTS'''
# from tsai.models.ResNet import ResNet
# from tsai.models import ResNet
# from tsai.models.ResNet import ResNet
# from fastai.metrics import accuracy
# from fastai.data.core import DataLoaders
# from fastai.learner import Learner
# import torch

# class FCMAEClassifier(nn.Module):
#     def __init__(self, encoder):
#         super().__init__()
#         self.encoder = encoder
#         self.classifier = ResNet(2048, 2)  # 2048 channels from encoder, 2 output classes

#     def forward(self, x):
#         with torch.no_grad():  # freeze encoder
#             x = self.encoder(x)
#         return self.classifier(x)

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Wrap your DataLoaders and set device
# dls = DataLoaders.from_dsets(
#     train_dataset,
#     test_dataset,
#     bs=32,
#     shuffle=True,
#     num_workers=0 
# )

# model = FCMAEClassifier(encoder)
# model.to(device)

# # Calculate class weights
# class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(train_labels), y=train_labels)
# class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32).to(device)

# # Use weighted loss
# loss_func = torch.nn.CrossEntropyLoss(weight=class_weights_tensor)

# # Replace your loss function in the learner
# learn = Learner(dls, model, loss_func=loss_func, metrics=accuracy)
# learn.fit_one_cycle(20, 1e-6)

# learn.recorder.plot_loss()  # Plot loss curves (training and validation loss)
# import matplotlib.pyplot as plt
# import seaborn as sns
# from sklearn.metrics import confusion_matrix
# import torch

# # Get predictions and true labels
# preds, targs = learn.get_preds(dl=learn.dls.valid)

# # Convert predictions to class labels (argmax for multi-class classification)
# pred_labels = preds.argmax(dim=1)

# # Compute confusion matrix
# cm = confusion_matrix(targs, pred_labels)

# # Plot confusion matrix
# plt.figure(figsize=(8, 6))
# sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False, 
#             xticklabels=['Class 0', 'Class 1'], yticklabels=['Class 0', 'Class 1'])
# plt.title('Confusion Matrix')
# plt.xlabel('Predicted Label')
# plt.ylabel('True Label')
# plt.show()

# from sklearn.metrics import (
#     accuracy_score,
#     precision_score,
#     recall_score,
#     f1_score,
#     average_precision_score,
#     roc_auc_score
# )

# # Convert to numpy arrays if needed
# true_labels = targs.cpu().numpy()
# pred_labels = pred_labels.cpu().numpy()
# pred_probs = preds[:, 1].cpu().numpy()  # Probability of class 1

# # Compute metrics
# accuracy = accuracy_score(true_labels, pred_labels)
# precision = precision_score(true_labels, pred_labels, zero_division=0)
# recall = recall_score(true_labels, pred_labels, zero_division=0)
# f1 = f1_score(true_labels, pred_labels, zero_division=0)
# ap = average_precision_score(true_labels, pred_probs)
# auc = roc_auc_score(true_labels, pred_probs)

# # Print results
# print(f"Accuracy: {accuracy:.4f}")
# print(f"Precision: {precision:.4f}")
# print(f"Recall: {recall:.4f}")
# print(f"F1 Score: {f1:.4f}")
# print(f"Average Precision (AP): {ap:.4f}")
# print(f"Area Under ROC Curve (AUC): {auc:.4f}")

